In [33]:
import numpy as np
import copy
import tensorflow as tf
from sklearn import preprocessing
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from keras import backend as K
import os
import pandas as pd
import matplotlib.pyplot as plt
from numpy.random import seed
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import skfuzzy as fuzz
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc



In [96]:
data=pd.read_csv(r"D:\codes\project\rainfall_data.csv")

In [205]:

def discretization(data):
    
    
    num_levels = 1000

    # Generate membership functions for each variable and level
    tpw_levels = np.arange(num_levels)
    tpw_mf = [fuzz.trimf(tpw_levels, [i-1, i, i+1]) for i in tpw_levels]

    rainfall_levels = np.arange(num_levels)
    rainfall_mf = [fuzz.trimf(rainfall_levels, [i-1, i, i+1]) for i in rainfall_levels]

    windspeed_levels = np.arange(num_levels)
    windspeed_mf = [fuzz.trimf(windspeed_levels, [i-1, i, i+1]) for i in windspeed_levels]

    # Apply fuzzy membership function to the data
    tpw_fuzzy_membership = [fuzz.interp_membership(tpw_levels, m, data["tpw"]) for m in tpw_mf]
    rainfall_fuzzy_membership = [fuzz.interp_membership(rainfall_levels, m, data["rainfall"]) for m in rainfall_mf]
    windspeed_fuzzy_membership = [fuzz.interp_membership(windspeed_levels, m, data["windspeed"]) for m in windspeed_mf]

    # Assign discretization levels based on the highest membership value
    tpw_discretized = np.argmax(tpw_fuzzy_membership, axis=0)
    rainfall_discretized = np.argmax(rainfall_fuzzy_membership, axis =0)
    windspeed_discretized =np.argmax(windspeed_fuzzy_membership ,axis=0)

    # Add the discretized column to the dataframe
    data["windspeed_discretized"] = windspeed_discretized
    data["tpw_discretized"] = tpw_discretized
    data["rainfall_discretized"] = rainfall_discretized 
    return data

data=discretization(data)
extreme_data=data[data["rainfall"]>50]
normal_data=data[data["rainfall"]<=50]

In [218]:
normal_data.iloc[:,1:]

,windspeed,tpw,rainfall,windspeed_discretized,tpw_discretized,rainfall_discretized,predicted_windspeed,predicted_tpw,predicted_rainfall,mse_loss
0,12.245595,0.033285,4.688589,12,0,5,12.245398,0.033942,4.679659,0.000080
1,12.825491,0.044874,11.330589,13,0,11,12.826557,0.051164,11.318683,0.000182
2,12.921664,0.010243,6.894713,13,0,7,12.921803,0.040150,6.884366,0.001002
3,15.149001,0.036881,4.613324,15,0,5,15.148251,0.038318,4.602582,0.000118
4,18.495907,0.139491,8.111635,18,0,8,18.495235,0.053082,8.097809,0.007658
...,...,...,...,...,...,...,...,...,...,...
2557,5.577215,0.009963,4.467977,6,0,4,5.578208,0.022783,4.463301,0.000187
2558,5.184293,0.002341,1.572182,5,0,2,5.184761,0.015031,1.568882,0.000172
2559,4.469007,0.000867,1.819019,4,0,2,4.469659,0.014504,1.816071,0.000195
2560,4.259090,0.001416,2.198017,4,0,2,4.259859,0.015107,2.195055,0.000197


In [159]:
def weighted_loss(normal_weight=1, extreme_weight=0.0001):
    def loss(y_true, y_pred):
        normal_loss = K.mean(K.square(y_true - y_pred))
        extreme_loss = K.mean(K.square(y_true - y_pred) * K.cast(y_true > 50, 'float32'))
        return K.mean(normal_weight * normal_loss + extreme_weight * extreme_loss)
    return loss

In [206]:
def find_best_reg_strength(train_data, reg_strengths, num_epochs, batch_size):
    best_reg_strength = None
    best_loss = float('inf')
    for reg_strength in reg_strengths:
        # Define the autoencoder architecture
        input_layer = Input(shape=(3,))
        encoded = Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(l2=reg_strength))(input_layer)
        decoded = Dense(3, activation=None)(encoded)

        # Create the autoencoder
        autoencoder = Model(input_layer, decoded)

        # Compile the autoencoder with the weighted loss function
        #autoencoder.compile(optimizer='adam', loss="mse")
        autoencoder.compile(optimizer='adam', loss=weighted_loss())
        # Train the model
        history = autoencoder.fit(train_data, train_data, epochs=num_epochs, batch_size=batch_size, validation_split=0.2, verbose=0)

        # Calculate the validation loss
        val_loss = np.mean(history.history['val_loss'])

        # Update the best regularization strength and loss
        if val_loss < best_loss:
            best_reg_strength = reg_strength
            best_loss = val_loss

    print('Best regularization strength:', best_reg_strength)
    return best_reg_strength

In [207]:
reg_strengths = [0.1, 1, 10, 100]
num_epochs = 2000
batch_size = 32
train_data = normal_data[["windspeed_discretized","tpw_discretized","rainfall_discretized"]].values.astype("float32")
#train_data = normal_data[["windspeed","tpw","rainfall"]].values.astype("float32")

best_reg_strength = find_best_reg_strength(train_data, reg_strengths, num_epochs, batch_size)

# Define the autoencoder architecture with the best regularization strength
input_layer = Input(shape=(3,),name='input')
encoded = Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(l2=best_reg_strength),name='encoder')(input_layer)
decoded = Dense(3, activation=None, name='decoder')(encoded)

# Create the autoencoder
autoencoder = Model(input_layer, decoded)

# Compile the autoencoder with the weighted loss function
#autoencoder.compile(optimizer='adam', loss="mse")
autoencoder.compile(optimizer='adam', loss=weighted_loss())

# Train the model with the majority class
history = autoencoder.fit(train_data, train_data, epochs=num_epochs, batch_size=batch_size, validation_split=0.2)


Best regularization strength: 0.1
Epoch 1/2000
64/64 [==============================] - 1s 7ms/step - loss: 73.3487 - val_loss: 46.8230
Epoch 2/2000
64/64 [==============================] - 0s 3ms/step - loss: 25.7205 - val_loss: 11.6844
Epoch 3/2000
64/64 [==============================] - 0s 3ms/step - loss: 5.6224 - val_loss: 2.7901
Epoch 4/2000
64/64 [==============================] - 0s 3ms/step - loss: 1.8699 - val_loss: 1.8028
Epoch 5/2000
64/64 [==============================] - 0s 3ms/step - loss: 1.2988 - val_loss: 1.3600
Epoch 6/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.9908 - val_loss: 1.0344
Epoch 7/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.7723 - val_loss: 0.8011
Epoch 8/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.6198 - val_loss: 0.6409
Epoch 9/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.5163 - val_loss: 0.5344
Epoch 10/2000
64/64 [==============================] - 0s 3ms/

Epoch 82/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0827 - val_loss: 0.0813
Epoch 83/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0802 - val_loss: 0.0788
Epoch 84/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0778 - val_loss: 0.0765
Epoch 85/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0755 - val_loss: 0.0741
Epoch 86/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0732 - val_loss: 0.0719
Epoch 87/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0709 - val_loss: 0.0697
Epoch 88/2000
64/64 [==============================] - 0s 2ms/step - loss: 0.0687 - val_loss: 0.0675
Epoch 89/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0666 - val_loss: 0.0655
Epoch 90/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0645 - val_loss: 0.0633
Epoch 91/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0625 - val_los

64/64 [==============================] - 0s 3ms/step - loss: 0.0142 - val_loss: 0.0148
Epoch 163/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0142 - val_loss: 0.0140
Epoch 164/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0141 - val_loss: 0.0140
Epoch 165/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0139 - val_loss: 0.0141
Epoch 166/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0139 - val_loss: 0.0137
Epoch 167/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0137 - val_loss: 0.0141
Epoch 168/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0137 - val_loss: 0.0137
Epoch 169/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0135 - val_loss: 0.0135
Epoch 170/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0134 - val_loss: 0.0134
Epoch 171/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0133 - val_loss: 0.

64/64 [==============================] - 0s 3ms/step - loss: 0.0094 - val_loss: 0.0097
Epoch 243/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0093 - val_loss: 0.0094
Epoch 244/2000
64/64 [==============================] - 0s 2ms/step - loss: 0.0094 - val_loss: 0.0093
Epoch 245/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0093 - val_loss: 0.0092
Epoch 246/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0092 - val_loss: 0.0093
Epoch 247/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0092 - val_loss: 0.0095
Epoch 248/2000
64/64 [==============================] - 0s 2ms/step - loss: 0.0091 - val_loss: 0.0092
Epoch 249/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0091 - val_loss: 0.0090
Epoch 250/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0091 - val_loss: 0.0090
Epoch 251/2000
64/64 [==============================] - 0s 2ms/step - loss: 0.0090 - val_loss: 0.

64/64 [==============================] - 0s 3ms/step - loss: 0.0071 - val_loss: 0.0071
Epoch 323/2000
64/64 [==============================] - 0s 2ms/step - loss: 0.0072 - val_loss: 0.0129
Epoch 324/2000
64/64 [==============================] - 0s 2ms/step - loss: 0.0082 - val_loss: 0.0073
Epoch 325/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0071 - val_loss: 0.0071
Epoch 326/2000
64/64 [==============================] - 0s 2ms/step - loss: 0.0070 - val_loss: 0.0069
Epoch 327/2000
64/64 [==============================] - 0s 2ms/step - loss: 0.0071 - val_loss: 0.0072
Epoch 328/2000
64/64 [==============================] - 0s 2ms/step - loss: 0.0070 - val_loss: 0.0069
Epoch 329/2000
64/64 [==============================] - 0s 2ms/step - loss: 0.0069 - val_loss: 0.0069
Epoch 330/2000
64/64 [==============================] - 0s 2ms/step - loss: 0.0070 - val_loss: 0.0070
Epoch 331/2000
64/64 [==============================] - 0s 2ms/step - loss: 0.0070 - val_loss: 0.

...

64/64 [==============================] - 0s 3ms/step - loss: 0.0028 - val_loss: 0.0028
Epoch 1675/2000
64/64 [==============================] - 0s 2ms/step - loss: 0.0028 - val_loss: 0.0084
Epoch 1676/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0067 - val_loss: 0.0060
Epoch 1677/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0044 - val_loss: 0.0030
Epoch 1678/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0028 - val_loss: 0.0029
Epoch 1679/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0029 - val_loss: 0.0028
Epoch 1680/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0028 - val_loss: 0.0028
Epoch 1681/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0028 - val_loss: 0.0028
Epoch 1682/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0028 - val_loss: 0.0028
Epoch 1683/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0028 - val

64/64 [==============================] - 0s 3ms/step - loss: 0.0030 - val_loss: 0.0027
Epoch 1754/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0029 - val_loss: 0.0027
Epoch 1755/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0027 - val_loss: 0.0027
Epoch 1756/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0027 - val_loss: 0.0029
Epoch 1757/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0029 - val_loss: 0.0030
Epoch 1758/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0041 - val_loss: 0.0029
Epoch 1759/2000
64/64 [==============================] - 0s 2ms/step - loss: 0.0040 - val_loss: 0.0045
Epoch 1760/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0032 - val_loss: 0.0028
Epoch 1761/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0027 - val_loss: 0.0027
Epoch 1762/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0027 - val

64/64 [==============================] - 0s 3ms/step - loss: 0.0027 - val_loss: 0.0029
Epoch 1833/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0027 - val_loss: 0.0027
Epoch 1834/2000
64/64 [==============================] - 0s 2ms/step - loss: 0.0027 - val_loss: 0.0027
Epoch 1835/2000
64/64 [==============================] - 0s 2ms/step - loss: 0.0027 - val_loss: 0.0028
Epoch 1836/2000
64/64 [==============================] - 0s 2ms/step - loss: 0.0040 - val_loss: 0.0080
Epoch 1837/2000
64/64 [==============================] - 0s 2ms/step - loss: 0.0037 - val_loss: 0.0031
Epoch 1838/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0031 - val_loss: 0.0043
Epoch 1839/2000
64/64 [==============================] - 0s 2ms/step - loss: 0.0028 - val_loss: 0.0027
Epoch 1840/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0027 - val_loss: 0.0027
Epoch 1841/2000
64/64 [==============================] - 0s 2ms/step - loss: 0.0027 - val

64/64 [==============================] - 0s 3ms/step - loss: 0.0026 - val_loss: 0.0026
Epoch 1912/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0026 - val_loss: 0.0026
Epoch 1913/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0026 - val_loss: 0.0027
Epoch 1914/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0026 - val_loss: 0.0026
Epoch 1915/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0027 - val_loss: 0.0026
Epoch 1916/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0027 - val_loss: 0.0027
Epoch 1917/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0027 - val_loss: 0.0026
Epoch 1918/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0026 - val_loss: 0.0026
Epoch 1919/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0026 - val_loss: 0.0026
Epoch 1920/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0026 - val

64/64 [==============================] - 0s 3ms/step - loss: 0.0026 - val_loss: 0.0026
Epoch 1991/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0026 - val_loss: 0.0026
Epoch 1992/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0026 - val_loss: 0.0026
Epoch 1993/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0026 - val_loss: 0.0026
Epoch 1994/2000
64/64 [==============================] - 0s 2ms/step - loss: 0.0026 - val_loss: 0.0026
Epoch 1995/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0026 - val_loss: 0.0026
Epoch 1996/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0027 - val_loss: 0.0028
Epoch 1997/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0028 - val_loss: 0.0026
Epoch 1998/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0028 - val_loss: 0.0026
Epoch 1999/2000
64/64 [==============================] - 0s 3ms/step - loss: 0.0026 - val

In [209]:
from tensorflow.keras.models import load_model

# Save the model to a file
autoencoder.save("my_autoencoder_model.h5")


In [169]:
test_data = data[["windspeed_discretized","tpw_discretized","rainfall_discretized"]].values
#test_data = data[["windspeed","tpw","rainfall"]].values
autoencoder = load_model("my_autoencoder_model.h5", custom_objects={"loss": weighted_loss()})

predicted_data = autoencoder.predict(test_data)

81/81 [==============================] - 0s 725us/step


In [28]:

# Calculate the Mean Squared Error (MSE) between the predicted outputs and the inputs for each sample
mse_loss = np.mean(np.power(test_data - predicted_data, 2), axis=1)

# Print the MSE for each sample
for i in range(len(mse_loss)):
    print('MSE for sample', i+1, ':', mse_loss[i])

MSE for sample 1 : 8.66475907900302e-07
MSE for sample 2 : 8.66475907900302e-07
MSE for sample 3 : 8.66475907900302e-07
MSE for sample 4 : 8.66475907900302e-07
MSE for sample 5 : 8.66475907900302e-07
MSE for sample 6 : 8.66475907900302e-07
MSE for sample 7 : 8.66475907900302e-07
MSE for sample 8 : 8.66475907900302e-07
MSE for sample 9 : 8.66475907900302e-07
MSE for sample 10 : 8.66475907900302e-07
MSE for sample 11 : 8.66475907900302e-07
MSE for sample 12 : 8.66475907900302e-07
MSE for sample 13 : 8.66475907900302e-07
MSE for sample 14 : 8.66475907900302e-07
MSE for sample 15 : 8.66475907900302e-07
MSE for sample 16 : 8.66475907900302e-07
MSE for sample 17 : 8.66475907900302e-07
MSE for sample 18 : 8.66475907900302e-07
MSE for sample 19 : 8.66475907900302e-07
MSE for sample 20 : 8.66475907900302e-07
MSE for sample 21 : 8.66475907900302e-07
MSE for sample 22 : 8.66475907900302e-07
MSE for sample 23 : 8.66475907900302e-07
MSE for sample 24 : 8.66475907900302e-07
MSE for sample 25 : 8.664

In [31]:
mse_loss.max()

0.4267334438791405

In [32]:
extreme_data

,date,windspeed,tpw,rainfall,windspeed_discretized,tpw_discretized,rainfall_discretized
2387,8/9/2019,21.575483,0.042829,80.636074,0,0,0
2388,8/10/2019,20.276800,0.043114,58.431061,0,0,0


In [241]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.models import load_model
from tensorflow.keras.models import load_model

# Define the custom loss function

import tensorflow.keras.backend as K

def weighted_loss(normal_weight=1, extreme_weight=0.001):
    def loss(y_true, y_pred):
        normal_loss = K.mean(K.square(y_true - y_pred))
        extreme_loss = K.mean(K.square(y_true - y_pred) * K.cast(y_true > 50, 'float32'))
        return K.mean(normal_weight * normal_loss + extreme_weight * extreme_loss)
    return loss

# Load your autoencoder model
autoencoder = load_model("my_autoencoder_model.h5", custom_objects={"loss": weighted_loss()})
#autoencoder = load_model("my_autoencoder_model.h5")
# Create a Keras model to get the output of the encoder and decoder layers
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer("encoder").output)
decoder = Model(inputs=autoencoder.get_layer("decoder").input, outputs=autoencoder.output)

# Get the encoded representation of the input data
encoded_X = encoder.predict(test_data)

# Reconstruct the input data from the encoded representation
reconstructed_X = decoder.predict(encoded_X)

# Compute the MSE reconstruction loss for each sample
mse_loss = np.sum((abs(test_data - reconstructed_X)), axis=1)
data["mse_loss"]=mse_loss
# Print the MSE reconstruction loss for each sample
print(mse_loss)
top10_indices = mse_loss.argsort()[::-1][:10]

# Print the MSE reconstruction loss for each of the top 10 samples
for i in top10_indices:
    print('MSE for sample', i+1, ':', mse_loss[i])
sample_idx = 2387  # 0-based index of sample 5
print('MSE loss for sample 2387:', mse_loss[sample_idx])
sample_idx = 2388  # 0-based index of sample 5
print('MSE loss for sample 2388:', mse_loss[sample_idx])

81/81 [==============================] - 0s 1ms/step
[0.03854451 0.05245662 0.01647256 ... 0.00445061 0.00409635 0.01155235]
MSE for sample 2366 : 0.7310782678453088
MSE for sample 876 : 0.6135539302450567
MSE for sample 1344 : 0.5656110189809418
MSE for sample 129 : 0.5491616541612087
MSE for sample 128 : 0.5101943885985947
MSE for sample 1232 : 0.4960464721078872
MSE for sample 1343 : 0.49569952394291644
MSE for sample 1434 : 0.4938348716598127
MSE for sample 2085 : 0.464778116557004
MSE for sample 1231 : 0.45835560537503045
MSE loss for sample 2387: 0.07576001327003609
MSE loss for sample 2388: 0.06858293039454672


In [226]:
from sklearn.metrics import mean_squared_error
mse_all = mean_squared_error(test_data, reconstructed_X)
import keras.backend as K

def weighted_loss(y_true, y_pred, weight=0.9):
    mse = K.mean(K.square(y_pred - y_true), axis=-1)
    normal_mask = y_true[:,0] == 0
    normal_mse = K.mean(mse[normal_mask])
    extreme_mse = K.mean(mse[~normal_mask])
    return (weight * normal_mse) + ((1 - weight) * extreme_mse)

autoencoder = load_model("my_autoencoder_model.h5", custom_objects={"loss": weighted_loss})
#autoencoder = load_model("my_autoencoder_model.h5")
# Create a Keras model to get the output of the encoder and decoder layers
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer("encoder").output)
decoder = Model(inputs=autoencoder.get_layer("decoder").input, outputs=autoencoder.output)

# Get the encoded representation of the normal and extreme rainfall data separately
normal_data_ = normal_data.iloc[:,1:4].astype('float32')
extreme_data_ = extreme_data.iloc[:,1:4].astype('float32')
encoded_normal = encoder.predict(normal_data_)
encoded_extreme = encoder.predict(extreme_data_)

# Reconstruct the normal and extreme rainfall data separately from the encoded representation
reconstructed_normal = decoder.predict(encoded_normal)
reconstructed_extreme = decoder.predict(encoded_extreme)

# Calculate the MSE between the original and reconstructed normal and extreme rainfall data separately
mse_normal = mean_squared_error(normal_data_, reconstructed_normal)
mse_extreme = mean_squared_error(extreme_data_, reconstructed_extreme)

# Print the MSE for all data and the separate MSE for normal and extreme rainfall data
print("MSE for all data: ", mse_all)
print("MSE for normal rainfall data: ", mse_normal)
print("MSE for extreme rainfall data: ", mse_extreme)

1/1 [==============================] - 0s 62ms/step
MSE for all data:  0.0020972070199253598
MSE for normal rainfall data:  0.0020225083
MSE for extreme rainfall data:  0.0006041576


In [165]:
data['predicted_windspeed'] = predicted_data[:,0]
data['predicted_tpw'] = predicted_data[:,1]
data['predicted_rainfall'] = predicted_data[:,2]


In [199]:
data.iloc[2365]


date                     7/18/2019
windspeed                20.137997
tpw                       0.626112
rainfall                  2.554412
windspeed_discretized           20
tpw_discretized                  1
rainfall_discretized             3
predicted_windspeed      20.013649
predicted_tpw             0.048848
predicted_rainfall        2.643523
mse_loss                  0.356637
Name: 2365, dtype: object

In [190]:
data.iloc[2387]


date                      8/9/2019
windspeed                21.575483
tpw                       0.042829
rainfall                 80.636074
windspeed_discretized           22
tpw_discretized                  0
rainfall_discretized            81
predicted_windspeed       22.88205
predicted_tpw             0.103391
predicted_rainfall       79.016685
mse_loss                  4.333204
Name: 2387, dtype: object

In [204]:
loss=(np.power(abs(test_data - reconstructed_X),2))
loss[2365]

array([0.01546243, 0.33323381, 0.00794081])

In [83]:
autoencoder.summary()

Model: "model_49"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 2)]               0         
                                                                 
 encoder (Dense)             (None, 16)                48        
                                                                 
 decoder (Dense)             (None, 2)                 34        
                                                                 
Total params: 82
Trainable params: 82
Non-trainable params: 0
_________________________________________________________________


In [40]:

data['predicted_windspeed'] = predicted_data[:,0]
data['predicted_tpw'] = predicted_data[:,1]
# Calculate the error between the predicted and actual rainfall values
data['error'] = np.abs(data['predicted_rainfall'] - data['rainfall'])

mean_error = np.mean(data['error'])
threshold = mean_error
print(threshold)

data['rainfall_class'] = np.where(data['error'] > threshold, 'Extreme', 'Normal')
# Define the actual rainfall class based on the threshold of 11
data['actual_rainfall_class'] = np.where(data['rainfall'] >50 , 'Extreme', 'Normal')

def find_roc_curve():
    data['actual_rainfall_binary'] = (data['rainfall_class'] == 'Extreme') & (data['actual_rainfall_class'] == 'Extreme')

    fpr, tpr, thresholds = roc_curve(data['actual_rainfall_binary'], data['error'])
    roc_auc = auc(fpr, tpr)
    # Calculate TNR
    tnr = 1 - fpr

    # Calculate Youden's J statistic for each threshold
    j_stat = tpr + tnr - 1

    # Find the index of the threshold that maximizes J
    best_threshold_idx = np.argmax(j_stat)

    # Get the best threshold
    best_threshold = thresholds[best_threshold_idx]
    threshold=round(best_threshold , 1)
    return threshold

threshold=round(find_roc_curve() , 1)
data['rainfall_class'] = np.where(data['error'] > threshold, 'Extreme', 'Normal')

# Define the actual rainfall class based on the threshold of 11
data['actual_rainfall_class'] = np.where(data['rainfall'] >50 , 'Extreme', 'Normal')


# Calculate the accuracy
num_correct = len(data[data['rainfall_class'] == data['actual_rainfall_class']])
num_total = len(data)
accuracy = num_correct / num_total * 100
print('Accuracy: {:.2f}%'.format(accuracy))

KeyError: 'predicted_rainfall'

In [ ]:
wrong_predictions = data[data['rainfall_class'] != data['actual_rainfall_class']]

# Print the wrong predictions
print(wrong_predictions)

In [42]:
data[data["rainfall"]>50]

,date,windspeed,tpw,rainfall,windspeed_discretized,tpw_discretized,rainfall_discretized,predicted_windspeed,predicted_tpw
2387,8/9/2019,21.575483,0.042829,80.636074,0,0,0,-0.00665,0.026587
2388,8/10/2019,20.276800,0.043114,58.431061,0,0,0,-0.00665,0.026587
